In [1]:
import json
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import numpy as np
with open('/home/xiaoguzai/数据/天池比赛地址相关数据集/Xeon3NLP_round1_train_20210524.json','r') as load_f:
    load_dict = json.load(load_f)
    print(load_dict[0:10])

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
[{'text_id': 'e225b9fd36b8914f42c188fc92e8918f', 'query': '河南省巩义市新华路街道办事处桐和街6号钢苑新区3号楼一单元', 'candidate': [{'text': '巩义市桐和街', 'label': '不匹配'}, {'text': '桐和街依家小店', 'label': '不匹配'}, {'text': '桐和街CHANG六LIULIU', 'label': '不匹配'}, {'text': '桐和街佳乐钢琴', 'label': '不匹配'}, {'text': '世博领秀城南门桐和街囍饭食堂', 'label': '不匹配'}]}, {'text_id': 'b2418ead7b48db4c09caa2934843c1b4', 'query': '老垅坡高家组省建五公司', 'candidate': [{'text': '高家巷省建五公司岳阳分公司', 'label': '完全匹配'}, {'text': '建设北路346号省建5公司', 'label': '不匹配'}, {'text': '老垅坡路西100米省建三公司(岳阳分公司)', 'label': '不匹配'}, {'text': '寇庄西路101号省建五公司', 'label': '不匹配'}, {'text': '卓刀泉南路21号省五建公司省建五公司', 'label': '不匹配'}]}, {'text_id': '5fa94565eb53463fa94ece56e8356fdc', 'query': '西关人和路工商银行对过腾信医药一楼眼镜店', 'candidate': [{'text': '河门口北街33号中国工商银行(河门口支行)', 'label': '不匹配'}, {'text': '河门口北街33号中国工商银行

读取数据时的注意点：开头要有[]，每一部分中间要有逗号，最后一波中间不要有逗号

In [2]:
print('label[0] question = ')
print(load_dict[0]['query'])
print('label[0] candidate = ')
print(load_dict[0]['candidate'])

label[0] question = 
河南省巩义市新华路街道办事处桐和街6号钢苑新区3号楼一单元
label[0] candidate = 
[{'text': '巩义市桐和街', 'label': '不匹配'}, {'text': '桐和街依家小店', 'label': '不匹配'}, {'text': '桐和街CHANG六LIULIU', 'label': '不匹配'}, {'text': '桐和街佳乐钢琴', 'label': '不匹配'}, {'text': '世博领秀城南门桐和街囍饭食堂', 'label': '不匹配'}]


In [3]:
nezha_ckpt_dir="/home/xiaoguzai/下载/NEZHA-Base-WWM/"
nezha_ckpt_file = nezha_ckpt_dir + "model.ckpt-691689"
nezha_config_file = nezha_ckpt_dir + "bert_config.json"

In [4]:
from tokenization import FullTokenizer
from tqdm import tqdm
import os
tokenizer = FullTokenizer(vocab_file=os.path.join(nezha_ckpt_dir,"vocab.txt"))
question1_id,question2_id = [],[]
question1_segment,question2_segment = [],[]
question_segment = []
segment_id = []
question_text = []
label_id = []

In [5]:
zero_number = 0
first_number = 0
second_number = 0
max_seq_len = 128
question_id = []
for data in load_dict:
    text1 = data['query']
    token1 = tokenizer.tokenize(text1)
    token1 = ["[CLS]"]+token1
    token_id1 = tokenizer.convert_tokens_to_ids(token1)
    if len(token_id1) > max_seq_len-1:
        token_id1 = token_id1[:max_seq_len-1]
    token_id1 = token_id1+tokenizer.convert_tokens_to_ids(["[SEP]"])
    question1_segment = [0]*len(token_id1)
    for data1 in data['candidate']:
        text2 = data1['text']
        token2 = tokenizer.tokenize(text2)
        token2 = token2
        token_id2 = tokenizer.convert_tokens_to_ids(token2)
        if len(token_id2) > max_seq_len-1:
            token_id2 = token_id2[:max_seq_len-1]
        token_id2 = token_id2+tokenizer.convert_tokens_to_ids(["[SEP]"])
        question2_segment = [1]*len(token_id2)
        token_id = token_id1+token_id2
        question_segment = question1_segment+question2_segment
        segment_id.append(question_segment)
        question_id.append(token_id)
        question_text.append(text1+' '+text2)
        if data1['label'] == '不匹配':
            label_id.append(0)
            zero_number = zero_number+1
        elif data1['label'] == '部分匹配':
            label_id.append(1)
            first_number = first_number+1
        else:
            label_id.append(2)
            second_number = second_number+1


weight0 = zero_number/(zero_number+first_number+second_number)
weight1 = first_number/(zero_number+first_number+second_number)
weight2 = second_number/(zero_number+first_number+second_number)
print('origin weight0 = ')
print(weight0)
print('origin weight1 = ')
print(weight1)
print('origin weight2 = ')
print(weight2)
weight0 = 1/weight0
weight1 = 1/weight1
weight2 = 1/weight2
totalweight = weight0+weight1+weight2
weight0 = weight0/totalweight
weight1 = weight1/totalweight
weight2 = weight2/totalweight
print('weight0 = ')
print(weight0)
print('weight1 = ')
print(weight1)
print('weight2 = ')
print(weight2)

origin weight0 = 
0.6069609947834023
origin weight1 = 
0.3307405909378468
origin weight2 = 
0.06229841427875092
weight0 = 
0.07950416546027425
weight1 = 
0.14590264599926467
weight2 = 
0.7745931885404611


In [6]:
print('question_id = ')
print(question_id[0:10])
print('label_id = ')
print(label_id[0:10])

question_id = 
[[101, 3777, 1298, 4689, 2343, 721, 2356, 3173, 1290, 6662, 6125, 6887, 1215, 752, 1905, 3432, 1469, 6125, 127, 1384, 7167, 5723, 3173, 1277, 124, 1384, 3517, 671, 1296, 1039, 102, 2343, 721, 2356, 3432, 1469, 6125, 102], [101, 3777, 1298, 4689, 2343, 721, 2356, 3173, 1290, 6662, 6125, 6887, 1215, 752, 1905, 3432, 1469, 6125, 127, 1384, 7167, 5723, 3173, 1277, 124, 1384, 3517, 671, 1296, 1039, 102, 3432, 1469, 6125, 898, 2157, 2207, 2421, 102], [101, 3777, 1298, 4689, 2343, 721, 2356, 3173, 1290, 6662, 6125, 6887, 1215, 752, 1905, 3432, 1469, 6125, 127, 1384, 7167, 5723, 3173, 1277, 124, 1384, 3517, 671, 1296, 1039, 102, 3432, 1469, 6125, 11680, 1063, 12306, 8636, 8207, 102], [101, 3777, 1298, 4689, 2343, 721, 2356, 3173, 1290, 6662, 6125, 6887, 1215, 752, 1905, 3432, 1469, 6125, 127, 1384, 7167, 5723, 3173, 1277, 124, 1384, 3517, 671, 1296, 1039, 102, 3432, 1469, 6125, 881, 727, 7167, 4433, 102], [101, 3777, 1298, 4689, 2343, 721, 2356, 3173, 1290, 6662, 6125, 6887, 121

In [7]:
with open(nezha_config_file,'r') as load_f:
    config = json.load(load_f)
print('config = ')
print(config)
config['embedding_size'] = config['hidden_size']
config['num_layers'] = config['num_hidden_layers']

config = 
{'attention_probs_dropout_prob': 0.1, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 21128, 'use_relative_position': True}


In [8]:
from nezha import Bert
batch_size = 48
bertmodel = Bert(maxlen=max_seq_len,batch_size=batch_size,**config)
input_ids1 = keras.layers.Input(shape=(None,),dtype='int32',name="token_ids1")
input_ids2 = keras.layers.Input(shape=(None,),dtype='int32',name="segment_ids1")
output = bertmodel([input_ids1,input_ids2])
#对应两个输入[token_ids1,segment_ids1]

#shape1 = output1.get_shape().as_list()
#shape2 = output2.get_shape().as_list()
output = keras.layers.Lambda(lambda seq: seq[:,0,:])(output)
print('output = ')
print(output)
output = keras.layers.Dropout(0.2)(output)
#!!!这里取出0而不取出-1是因为最后一位可能有填充数值
#output1 = tf.reshape(output1,[batch_size,max_seq_len*shape1[2]])
#output2 = tf.reshape(output2,[batch_size,max_seq_len*shape1[2]])
#原先这里的dropout=0.5,现在的dropout=0.1
output = keras.layers.Dense(units=768,activation="tanh")(output)
output = keras.layers.Dropout(0.2)(output)
output = keras.layers.Dense(units=3,activation="softmax")(output)
r"""
class CrossEntropy(tf.keras.layers.Layer):
    def call(self,inputs):
        loss = K.sparse_categorical_crossentropy(tf.convert_to_tensor(label_id),inputs)
        print('label_id = ')
        print(tf.convert_to_tensor(label_id))
        print('inputs = ')
        print(inputs)
        print('loss = ')
        print(loss)
        self.add_loss(loss)
        return inputs
crossentropy = CrossEntropy()
output = crossentropy(output)
"""
print('after Dense')
print('output = ')
print(output)
#这里不能使用CrossEntropy的原因在于label_id之中无法取出每一波的批次值，所以选择使用loss函数
model = keras.Model(inputs=[input_ids1,input_ids2],outputs=output)
#model = keras.Model(inputs=input_ids1,outputs=output)
#sparse_categorical_accuracy:检查y_true中的值与y_pred中最大值对应的index是否相等

#之前learning_rate = 0.000001,现在learning_rate = 0.00001
#from_logits = False输入已经符合某种分布，系统只会把你概率归一化
#from_logits = True输入的是原始数据，系统会帮你softmax之后再进行计算
#后面指定学习率了，这里的adam就不用指定学习率了
model.summary()

__init__ Nezha
with call_context.enter
with call_context.enter
output = 
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='lambda/strided_slice:0', description="created by layer 'lambda'")
with call_context.enter
with call_context.enter
with call_context.enter
with call_context.enter
after Dense
output = 
KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), name='dense_1/Softmax:0', description="created by layer 'dense_1'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
token_ids1 (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
segment_ids1 (InputLayer)       [(None, None)]       0                                          

In [9]:
from nezha_loader import load_nezha_stock_weights
load_nezha_stock_weights(nezha=bertmodel,ckpt_path=nezha_ckpt_file)

load_nezha_stock_weights
Done loading 196 NEZHA weights from: /home/xiaoguzai/下载/NEZHA-Base-WWM/model.ckpt-691689 into <nezha.Bert object at 0x7fe12c101ee0>. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bad_steps
	bert/embeddings/LayerNorm/beta/lamb_m
	bert/embeddings/LayerNorm/beta/lamb_v
	bert/embeddings/LayerNorm/gamma/lamb_m
	bert/embeddings/LayerNorm/gamma/lamb_v
	bert/embeddings/token_type_embeddings/lamb_m
	bert/embeddings/token_type_embeddings/lamb_v
	bert/embeddings/word_embeddings/lamb_m
	bert/embeddings/word_embeddings/lamb_v
	bert/encoder/layer_0/attention/output/LayerNorm/beta/lamb_m
	bert/encoder/layer_0/attention/output/LayerNorm/beta/lamb_v
	bert/encoder/layer_0/attention/output/LayerNorm/gamma/lamb_m
	bert/encoder/layer_0/attention/output/LayerNorm/gamma/lamb_v
	bert/encoder/layer_0/attention/output/dense/bias/lamb_m
	bert/encoder/layer_0/attention/output/dense/bias/lamb_v
	bert/enc

[]

In [10]:
from sklearn.model_selection import KFold
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
result_point = []
test_point = []
#kf = KFold(n_splits=5,shuffle=True)
def create_learning_rate_scheduler(max_learn_rate=5e-5,
                                   end_learn_rate=1e-7,
                                   warmup_epoch_count=10,
                                   total_epoch_count=90):
    print('create_learning_rate_scheduler')
    def lr_scheduler(epoch):
    #18年Facebook提出的gradual warmup刚开始训练的时候，模型的权重是随机
    #初始化的，此时若选择一个较大的学习率，可能带来模型的不确定(振荡),选择warm
    #up学习率的方式，可以使得开始训练的几个epoches或者一些steps学习率较小
    #在预热的小学习率下，模型可以慢慢趋于稳定，等模型相对稳定后再选择预先设置学习率
        if epoch < warmup_epoch_count:
            res = (max_learn_rate/warmup_epoch_count) * (epoch + 1)
        else:
            res = max_learn_rate*math.exp(math.log(end_learn_rate/max_learn_rate)*(epoch-warmup_epoch_count+1)/(total_epoch_count-warmup_epoch_count+1))
        return float(res)
    #Warmup不足之处在于从一个很小的学习率一下子变为比较大的学习率可能会导致训练误差突然增大
    #facebook提出了从最初的小学习率开始，每个step增大一点
    learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)
    #keras.callbacks.LearningRateScheduler(schedule,verbose=0)
    #schedule:接受epoch作为输入(整数，从0开始迭代)，然后返回一个学习率作为输出(浮点数)
    #verbose:0:安静,1:更新信息
    return learning_rate_scheduler
    
#split = StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=42)
#split = StratifiedShuffleSplit(n_splits=5,test_size=0.1,random_state=42)
#分成5组，每组中的test=0.1也就是说循环之中使用的for循环每次从5组之中取出一组数据
#所以之前一直在用一组数据进行训练
class DataGenerator(keras.callbacks.Callback):
    def __init__(self,token_ids,segment_ids,label_ids,batch_size=48,max_seq_len=128):
        self.token_ids = token_ids
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.batch_size = batch_size
        self.maxlen = max_seq_len
        self.totals = len(self.token_ids)
        self.best_score = 0
        self.train_token_ids = None
        self.test_token_ids = None
        self.train_segment_ids = None
        self.test_segment_ids = None
        self.train_label = None
        self.test_label = None
        #这个取出数值放在__init__函数之中，每次都是一样的数值，如果放在cycle
        #函数之中，每次取出来的数值都是不一样的
    
    def __len__(self):
        return int(np.floor(len(self.token_ids)/self.batch_size))

    def sample(self,random=False):
        #sample拆分出训练集进行训练
        indices = list(range(len(self.train_token_ids)))
        np.random.shuffle(indices)
        for i in indices:
            yield self.train_token_ids[i],self.train_segment_ids[i],self.train_label[i]
    
    def __iter__(self,random=False):
        #__iter__拆分出对应的batch
        random = False
        batch_token_ids = []
        batch_segment_ids = []
        batch_label_ids = []
        currents = 0
        batch_data = []
        for token_ids,segment_ids,label_ids in self.sample(random):
            if len(token_ids) > self.maxlen*2:
            #!!!注意这里要为self.maxlen*2,因为单个长度为128，两个加起来不能超过256
                token_ids = token_ids[:self.maxlen]
                segment_ids = segment_ids[:self.maxlen]
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_label_ids.append(label_ids)
            
            r"""
            k折交叉验证的时候就不加量了
            if label_ids == 2:
            #!!!标记部分
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_label_ids.append(label_ids)
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_label_ids.append(label_ids)
            """
            #实验表明只加两波效果比较好
            currents = currents+1
            if len(batch_token_ids) == self.batch_size or currents == self.totals:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                yield [np.array(batch_token_ids),np.array(batch_segment_ids)],np.array(batch_label_ids)
                #yield [np.array(batch_token_ids1),np.array(batch_segment_ids1)],np.array(batch_label_ids)
                batch_token_ids,batch_segment_ids,batch_label_ids = [],[],[]
                #这里必须要统一维度
                
    def cycle(self,random=True):
        #for index1,index2 in kf.split(self.token_ids1,self.token_ids1):
        flag = False
        split = StratifiedShuffleSplit(n_splits=5,test_size=0.1)
        #!!!这个split一定要定义在cycle之中，否则定义好了的random_state对应值一样
        for index1,index2 in split.split(self.token_ids,self.label_ids):
            self.train_token_ids = np.array(self.token_ids)[index1]
            self.test_token_ids = np.array(self.token_ids)[index2]
            self.train_segment_ids = np.array(self.segment_ids)[index1]
            self.test_segment_ids = np.array(self.segment_ids)[index2]
            self.train_label = np.array(self.label_ids)[index1]
            self.test_label = np.array(self.label_ids)[index2]
        while True:
            for d in self.__iter__(random):
                yield d
    
    def on_epoch_end(self,epoch,logs):
        zero_reality = 0
        partial_reality = 0
        true_reality = 0
        zero_predict = 0
        partial_predict = 0
        true_predict = 0
        zero_true = 0
        partial_true = 0
        true_true = 0
        predict_label = []
        data2 = []
        for i in tqdm(range(0,len(self.test_token_ids))):
            token_id = self.test_token_ids[i]
            segment_id = self.test_segment_ids[i]
            res = model.predict([np.array([token_id]),np.array([segment_id])]).argmax(axis=-1)
            data2.append(res[0])
            if res[0] == 0 and self.test_label[i] == 0:
                zero_true = zero_true+1
            elif res[0] == 1 and self.test_label[i] == 1:
                partial_true = partial_true+1
            elif res[0] == 2 and self.test_label[i] == 2:
                true_true = true_true+1
            predict_label.append(res[0])
        sets = Counter(self.test_label)
        zero_reality = sets[0]
        partial_reality = sets[1]
        true_reality = sets[2]
        zero_predict = predict_label.count(0)
        partial_predict = predict_label.count(1)
        true_predict = predict_label.count(2)
        
        if zero_predict == 0:
            zero_point1 = 0
        else:
            zero_point1 = zero_true/zero_predict   

        if zero_reality == 0:
            zero_point2 = 0
        else:
            zero_point2 = zero_true/zero_reality
            
        if zero_point1+zero_point2 == 0:
            zero_point3 = 0
        else:
            zero_point3 = 2*zero_point1*zero_point2/(zero_point1+zero_point2)
        
        if partial_predict == 0:
            partial_point1 = 0
        else:
            partial_point1 = partial_true/partial_predict

        if partial_reality == 0:
            partial_point2 = 0
        else:
            partial_point2 = partial_true/partial_reality

        if partial_point1+partial_point2 == 0:
            partial_point3 = 0
        else:
            partial_point3 = 2*partial_point1*partial_point2/(partial_point1+partial_point2)
        
        if true_predict == 0:
            true_point1 = 0
        else:
            true_point1 = true_true/true_predict

        if true_reality == 0:
            true_point2 = 0
        else:
            true_point2 = true_true/true_reality

        if true_point1+true_point2 == 0:
            true_point3 = 0
        else:
            true_point3 = 2*true_point1*true_point2/(true_point1+true_point2)
        
        total_score1 = (zero_point1+zero_point2+zero_point3)/3
        total_score2 = (partial_point1+partial_point2+partial_point3)/3
        total_score3 = (true_point1+true_point2+true_point3)/3
        final_score = total_score1+total_score2+total_score3
        final_score = final_score/3
        print('验证集')
        print('分类1 准确率')
        print(zero_point1)
        print('分类1 召回率')
        print(zero_point2)
        print('分类1 f1_score')
        print(zero_point3)
        print('分类2 准确率')
        print(partial_point1)
        print('分类2 召回率')
        print(partial_point2)
        print('分类2 f1_score')
        print(partial_point3)
        print('分类3 准确率')
        print(true_point1)
        print('分类3 召回率')
        print(true_point2)
        print('分类3 f1_score')
        print(true_point3)
        print('current final_score = ')
        print(final_score)
        result_point.append(final_score)
        data1 = self.test_label
        data1 = np.expand_dims(data1,axis=-1)
        data2 = predict_label
        data2 = np.expand_dims(data2,axis=-1)
        point = keras.metrics.sparse_categorical_accuracy(data1,data2)
        print('evaluate sparse_categorical_accuracy = ')
        print(sum(point)/len(predict_label))
        print('#######################################')
        if final_score > self.best_score:
            self.best_score = final_score
            print('self.best_score = ')
            print(self.best_score)
            model.save_weights('./天池地址匹配the_best_model.h5')
            #注意这里存的为epoch+5

In [11]:
def sequence_padding(inputs,padding = 0):
    length = max([len(x) for x in inputs])
    pad_width = [(0,0) for _ in np.shape(inputs[0])]
    outputs = []
    for x in inputs:
        x = x[:length]
        pad_width[0] = (0,length-len(x))
        x = np.pad(x,pad_width,'constant',constant_values=padding)
        outputs.append(x)
    return np.array(outputs)

In [12]:
#output_ids = model(input_ids1)
output_ids = model([input_ids1,input_ids2])

with call_context.enter


In [13]:
type(question1_id)

list

提前停止模型的使用方法：
```python
import keras
early_stopping=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0,
                              patience=0, verbose=0, mode='auto',
                              baseline=None, restore_best_weights=False)
model.fit(callbacks = [early_stopping])
```
这里的monitor可以换成为train_generator.best_score
如果best_score没有提升，就停止模型的寻俩

In [14]:
def compute_loss(y_true,y_pred):
    print('y_true = ')
    print(y_true)
    print('y_pred = ')
    print(y_pred)
    loss = K.sparse_categorical_crossentropy(y_true,y_pred)
    print('loss = ')
    print(loss)
    #result = np.sum(loss,axis=-1)/3
    #return loss[0]+2*loss[1]+5*loss[2]
    return loss

In [15]:
label_id[0:10]

[0, 0, 0, 0, 0, 2, 0, 0, 0, 0]

In [16]:
train_generator = DataGenerator(question_id,segment_id,label_id)
model.compile(#optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              optimizer=keras.optimizers.Adam(learning_rate=2e-5),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              #loss=compute_loss,
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

train_generator.on_epoch_end(epoch=1,logs=None)

model.fit(
    train_generator.cycle(),
    steps_per_epoch = len(train_generator),
    epochs = 10,
    callbacks = [train_generator]
)

In [17]:
#!!!!!!!!!!!两处需要改正的一处在create_learning_rate_schduler,一处在model.save_weights那个地方!!!!!!

model.load_weights('./天池地址匹配the_best_model.h5')

In [18]:
model.fit(
    train_generator.cycle(),
    steps_per_epoch = len(train_generator),
    epochs = 20,
    callbacks = [train_generator]
)

<ipython-input-10-4c972a307f22>:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.train_token_ids = np.array(self.token_ids)[index1]
<ipython-input-10-4c972a307f22>:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.test_token_ids = np.array(self.token_ids)[index2]
<ipython-input-10-4c972a307f22>:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when 

Epoch 1/20
  26/2008 [..............................] - ETA: 9:09 - loss: 0.8824 - acc: 0.5419

KeyboardInterrupt: 

In [ ]:
fp = open('/home/xiaoguzai/数据/天池比赛地址相关数据集/验证集错误部分.txt','w')
current_str = ['不匹配','部分匹配','完全匹配']
for i in tqdm(range(0,len(train_generator.test_token_ids))):
    token_id = train_generator.test_token_ids[i]
    segment_id = train_generator.test_segment_ids[i]
    res = model.predict([np.array([token_id]),np.array([segment_id])]).argmax(axis=-1)
    if res[0] != train_generator.test_label[i]:
        fp.write(str(i)+' '+question_text[i]+'\n')
        fp.write('my_label:'+current_str[res[0]])
        fp.write(' true_label:'+current_str[train_generator.test_label[i]]+'\n')
fp.close()
r"""
data2.append(res[0])
if res[0] == 0 and train_generator.test_label[i] == 0:
    zero_true = zero_true+1
elif res[0] == 1 and train_generator.test_label[i] == 1:
    partial_true = partial_true+1
elif res[0] == 2 and train_generator.test_label[i] == 2:
    true_true = true_true+1
predict_label.append(res[0])
"""

model.load_weights('./天池地址匹配the_best_model.h5')

In [ ]:
with open('/home/xiaoguzai/数据/天池比赛地址相关数据集/Xeon3NLP_round1_test_20210524.json','r') as load_f:
    test_data = json.load(load_f)

model.save_weights('./天池地址匹配epoch=3.h5')

In [ ]:
#for data in test_data:
for i in tqdm(range(0,len(test_data))):
    data = test_data[i]
    text1 = data['query']
    token1 = tokenizer.tokenize(text1)
    token1 = ["[CLS]"]+token1
    token_id1 = tokenizer.convert_tokens_to_ids(token1)
    if len(token_id1) > max_seq_len-1:
        token_id1 = token_id1[:max_seq_len-1]
    token_id1 = token_id1+tokenizer.convert_tokens_to_ids(["[SEP]"])
    segment_id1 = [0]*len(token_id1)
    for j in range(len(data['candidate'])):
        data1 = data['candidate'][j]
        text2 = data1['text']
        token2 = tokenizer.tokenize(text2)
        token_id2 = tokenizer.convert_tokens_to_ids(token2)
        if len(token_id2) > max_seq_len-1:
            token_id2 = token_id2[:max_seq_len-1]
        token_id2 = token_id2+tokenizer.convert_tokens_to_ids(["[SEP]"])
        segment_id2 = [1]*len(token_id2)
        current = [token_id1+token_id2]
        res = model.predict([np.array([token_id1+token_id2]),np.array([segment_id1+segment_id2])]).argmax(axis=-1)
        if res[0] == 0:
            test_data[i]['candidate'][j]['label'] = '不匹配'
        elif res[0] == 1:
            test_data[i]['candidate'][j]['label'] = '部分匹配'
        else:
            test_data[i]['candidate'][j]['label'] = '完全匹配'

In [ ]:
#file_obj = open('/home/xiaoguzai/数据/天池比赛地址相关数据集/results3.txt','w',encoding='utf-8')
file_obj = open('nezha_提交结果_results.txt','w',encoding='utf-8')
for data in test_data:
    file_obj.write(json.dumps(data,ensure_ascii=False))
    file_obj.write('\n')
file_obj.close()

In [ ]:
file_obj = open('nezha计算结果1.txt','w',encoding='utf-8')
for index in range(len(test_data)):
    file_obj.write(str(index)+' '+str(test_data[i]['candidate'][j]['label'])+'\n')
file_obj.close()

In [ ]:
print(len(test_data))